In [2]:
#IPython magic to reload libraries
%load_ext autoreload
%autoreload 2

# Extra code so Juypter doesn't crash on mac os
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
import sys
sys.path.insert(0, 'utils')  # noqa

import numpy as np
import gym
import ntm
import interfaces

In [494]:
# Training Params
train_iter = 10000
batch_size = 1


In [479]:
# Task
env = gym.make('Copy-v0')
env = env.unwrapped
print(env.action_space)
print(env.observation_space)


Tuple(Discrete(2), Discrete(2), Discrete(5))
Discrete(6)


In [489]:
# Controller 
Controller = ntm.NTM()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tape_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
memory_input (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 11)           0           tape_input[0][0]                 
                                                                 memory_input[0][0]               
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 32)           384         concatenate_2[0][0]        

In [488]:
#Over Fit on single char
output = Controller.run(tape_input=1)
print(output)
Controller.update(output_target=4)

(3, 1, 1)
Loss:  [0.9380556]


In [495]:
output_control_cmds = ["no","yes"]
input_control_cmds = ["left","right"]
int_to_char = ["A","B","C","D","E","F"]
render_env = True
for i in range(train_iter):
    
    # Collect experince using current policy/controller
    for j in range(batch_size):
        
        episode_steps = 0
        observation = env.reset()
        if render_env: env.render()

        while True:
            episode_steps += 1
            
            #Run RL-NTM controller and get outputs
            outputs = Controller.run(tape_input=observation)

            output_content = outputs[0] # Which character to write (ignored if the above sub-action is 0) 
            output_control = outputs[1] #  Whether to write to the output tape ["no","yes"] = [0,1]
            input_control = outputs[2] # Direction to move the read head ["left","right"] = [0,1]
           
            actions = (input_control, output_control, output_content)
            
            print("Policy - obs: {} action: {}, {}, {}".format(int_to_char[observation],
                  int_to_char[output_content],
                  output_control_cmds[output_control],
                  input_control_cmds[input_control]))

            #Get output target before updating enviroment
            target = env.target[env.write_head_position]
            observation, reward, done, info = env.step(actions)
            if render_env: env.render()
                
            if output_control_cmds[output_control] == "yes":
                Controller.update(output_target=target)
                
            if done:
                done = False
                break
        

Policy - obs: E action: E, yes, right
Loss:  [0.76104033]
Policy - obs: A action: A, yes, right
Loss:  [0.56013846]
Policy - obs: C action: D, yes, right
Loss:  [1.3923239]
Policy - obs: B action: C, yes, right
Loss:  [0.8843392]
Policy - obs: E action: E, yes, right
Loss:  [0.59238297]
Policy - obs: D action: E, yes, right
Loss:  [1.2324259]
Policy - obs: D action: D, yes, right
Loss:  [1.0145679]
Policy - obs: B action: B, yes, right
Loss:  [0.8382191]
Policy - obs: C action: E, yes, right
Loss:  [1.4502378]
Policy - obs: E action: E, yes, right
Loss:  [0.7195665]
Policy - obs: D action: D, yes, right
Loss:  [1.0545746]
Policy - obs: A action: A, yes, right
Loss:  [0.5872635]
Policy - obs: A action: A, yes, right
Loss:  [0.51713824]
Policy - obs: D action: D, yes, right
Loss:  [0.8844441]
Policy - obs: A action: A, yes, right
Loss:  [0.47356987]
Policy - obs: B action: B, yes, right
Loss:  [0.9255749]
Policy - obs: A action: A, yes, right
Loss:  [0.46286374]
Policy - obs: E action: E

Policy - obs: C action: C, yes, right
Loss:  [0.9408477]
Policy - obs: C action: B, yes, right
Loss:  [0.8054725]
Policy - obs: E action: E, yes, right
Loss:  [0.34075886]
Policy - obs: E action: E, yes, right
Loss:  [0.23674093]
Policy - obs: E action: E, yes, right
Loss:  [0.18145642]
Policy - obs: C action: C, yes, right
Loss:  [0.76583827]
Policy - obs: E action: E, yes, right
Loss:  [0.17311072]
Policy - obs: A action: A, yes, right
Loss:  [0.47906584]
Policy - obs: D action: C, yes, right
Loss:  [1.6837521]
Policy - obs: A action: A, yes, right
Loss:  [0.36964828]
Policy - obs: E action: E, yes, right
Loss:  [0.2581321]
Policy - obs: C action: E, yes, right
Loss:  [0.7476883]
Policy - obs: A action: B, yes, right
Loss:  [0.32519442]
Policy - obs: D action: D, yes, right
Loss:  [1.4225093]
Policy - obs: A action: B, yes, right
Loss:  [0.27705857]
Policy - obs: C action: C, yes, right
Loss:  [0.7143726]
Policy - obs: D action: E, yes, right
Loss:  [1.1493824]
Policy - obs: E action

Loss:  [0.42054936]
Policy - obs: B action: A, yes, right
Loss:  [0.4253753]
Policy - obs: E action: D, yes, right
Loss:  [0.99479043]
Policy - obs: B action: A, yes, right
Loss:  [0.3881292]
Policy - obs: E action: E, yes, right
Loss:  [0.28984973]
Policy - obs: D action: D, yes, right
Loss:  [0.8525723]
Policy - obs: C action: E, yes, right
Loss:  [0.85507435]
Policy - obs: A action: A, yes, right
Loss:  [0.29351264]
Policy - obs: D action: D, yes, right
Loss:  [0.5891676]
Policy - obs: A action: A, yes, right
Loss:  [0.21317334]
Policy - obs: C action: D, yes, right
Loss:  [0.74054486]
Policy - obs: B action: A, yes, right
Loss:  [0.5127895]
Policy - obs: C action: D, yes, right
Loss:  [0.6324509]
Policy - obs: A action: B, yes, right
Loss:  [0.21979134]
Policy - obs: E action: E, yes, right
Loss:  [0.5958266]
Policy - obs: E action: E, yes, right
Loss:  [0.2189986]
Policy - obs: D action: E, yes, right
Loss:  [1.0138857]
Policy - obs: B action: B, yes, right
Loss:  [0.5000594]
Poli

Policy - obs: B action: C, yes, right
Loss:  [0.2991296]
Policy - obs: C action: B, yes, right
Loss:  [0.4195927]
Policy - obs: A action: A, yes, right
Loss:  [0.18634604]
Policy - obs: B action: A, yes, right
Loss:  [0.30430508]
Policy - obs: C action: E, yes, right
Loss:  [0.34616885]
Policy - obs: B action: B, yes, right
Loss:  [0.27903792]
Policy - obs: C action: C, yes, right
Loss:  [0.30939996]
Policy - obs: A action: A, yes, right
Loss:  [0.19814499]
Policy - obs: E action: E, yes, right
Loss:  [0.14494969]
Policy - obs: D action: E, yes, right
Loss:  [0.99832475]
Policy - obs: C action: C, yes, right
Loss:  [0.3298866]
Policy - obs: A action: A, yes, right
Loss:  [0.1417146]
Policy - obs: D action: C, yes, right
Loss:  [0.6247218]
Policy - obs: C action: D, yes, right
Loss:  [0.3453941]
Policy - obs: D action: D, yes, right
Loss:  [0.46111882]
Policy - obs: C action: C, yes, right
Loss:  [0.35166237]
Policy - obs: E action: C, yes, right
Loss:  [0.6300415]
Policy - obs: C actio

Policy - obs: B action: B, yes, right
Loss:  [0.25249553]
Policy - obs: B action: B, yes, right
Loss:  [0.20996757]
Policy - obs: C action: D, yes, right
Loss:  [0.2683071]
Policy - obs: D action: D, yes, right
Loss:  [0.7031806]
Policy - obs: A action: B, yes, right
Loss:  [0.09661528]
Policy - obs: D action: E, yes, right
Loss:  [0.3275484]
Policy - obs: B action: C, yes, right
Loss:  [0.20263332]
Policy - obs: E action: E, yes, right
Loss:  [0.42796177]
Policy - obs: A action: A, yes, right
Loss:  [0.1059732]
Policy - obs: C action: C, yes, right
Loss:  [0.31976134]
Policy - obs: A action: A, yes, right
Loss:  [0.07398624]
Policy - obs: E action: E, yes, right
Loss:  [0.08369398]
Policy - obs: B action: B, yes, right
Loss:  [0.23350784]
Policy - obs: D action: E, yes, right
Loss:  [0.63934094]
Policy - obs: B action: B, yes, right
Loss:  [0.18529832]
Policy - obs: C action: C, yes, right
Loss:  [0.32581368]
Policy - obs: D action: E, yes, right
Loss:  [0.3377217]
Policy - obs: B act

...

Policy - obs: B action: B, yes, right
Loss:  [0.00713912]
Policy - obs: E action: E, yes, right
Loss:  [0.00419093]
Policy - obs: C action: C, yes, right
Loss:  [0.01019044]
Policy - obs: D action: D, yes, right
Loss:  [0.00943851]
Policy - obs: A action: A, yes, right
Loss:  [0.00292394]
Policy - obs: B action: B, yes, right
Loss:  [0.00716601]
Policy - obs: B action: B, yes, right
Loss:  [0.00694349]
Policy - obs: A action: A, yes, right
Loss:  [0.00303269]
Policy - obs: B action: B, yes, right
Loss:  [0.00687093]
Policy - obs: C action: C, yes, right
Loss:  [0.01026095]
Policy - obs: C action: C, yes, right
Loss:  [0.00980826]
Policy - obs: B action: B, yes, right
Loss:  [0.00694698]
Policy - obs: C action: C, yes, right
Loss:  [0.0095641]
Policy - obs: A action: A, yes, right
Loss:  [0.00309834]
Policy - obs: E action: E, yes, right
Loss:  [0.00419033]
Policy - obs: E action: E, yes, right
Loss:  [0.00379674]
Policy - obs: A action: A, yes, right
Loss:  [0.00301045]
Policy - obs: C

Policy - obs: D action: D, yes, right
Loss:  [0.00908706]
Policy - obs: A action: A, yes, right
Loss:  [0.00311005]
Policy - obs: A action: A, yes, right
Loss:  [0.00297942]
Policy - obs: D action: D, yes, right
Loss:  [0.00857136]
Policy - obs: A action: A, yes, right
Loss:  [0.00286853]
Policy - obs: C action: C, yes, right
Loss:  [0.00890513]
Policy - obs: D action: D, yes, right
Loss:  [0.00831156]
Policy - obs: A action: A, yes, right
Loss:  [0.00276249]
Policy - obs: C action: C, yes, right
Loss:  [0.00875004]
Policy - obs: A action: A, yes, right
Loss:  [0.00266071]
Policy - obs: E action: E, yes, right
Loss:  [0.004976]
Policy - obs: E action: E, yes, right
Loss:  [0.0044205]
Policy - obs: B action: B, yes, right
Loss:  [0.00704481]
Policy - obs: D action: D, yes, right
Loss:  [0.00880729]
Policy - obs: D action: D, yes, right
Loss:  [0.0083353]
Policy - obs: C action: C, yes, right
Loss:  [0.00882852]
Policy - obs: C action: C, yes, right
Loss:  [0.00851713]
Policy - obs: A ac

Policy - obs: A action: A, yes, right
Loss:  [0.00270518]
Policy - obs: C action: C, yes, right
Loss:  [0.00678949]
Policy - obs: B action: B, yes, right
Loss:  [0.00698983]
Policy - obs: E action: E, yes, right
Loss:  [0.00414496]
Policy - obs: A action: A, yes, right
Loss:  [0.00269173]
Policy - obs: E action: E, yes, right
Loss:  [0.00376539]
Policy - obs: E action: E, yes, right
Loss:  [0.00343949]
Policy - obs: C action: C, yes, right
Loss:  [0.00671346]
Policy - obs: A action: A, yes, right
Loss:  [0.00260244]
Policy - obs: E action: E, yes, right
Loss:  [0.00320327]
Policy - obs: B action: B, yes, right
Loss:  [0.00720083]
Policy - obs: B action: B, yes, right
Loss:  [0.00694481]
Policy - obs: E action: E, yes, right
Loss:  [0.0030398]
Policy - obs: E action: E, yes, right
Loss:  [0.00282591]
Policy - obs: C action: C, yes, right
Loss:  [0.0067131]
Policy - obs: E action: E, yes, right
Loss:  [0.00265826]
Policy - obs: E action: E, yes, right
Loss:  [0.00250767]
Policy - obs: D 

Policy - obs: B action: B, yes, right
Loss:  [0.00723884]
Policy - obs: D action: D, yes, right
Loss:  [0.0073382]
Policy - obs: D action: D, yes, right
Loss:  [0.00702188]
Policy - obs: B action: B, yes, right
Loss:  [0.00694836]
Policy - obs: E action: E, yes, right
Loss:  [0.00496438]
Policy - obs: A action: A, yes, right
Loss:  [0.00219974]
Policy - obs: D action: D, yes, right
Loss:  [0.00703641]
Policy - obs: B action: B, yes, right
Loss:  [0.00681638]
Policy - obs: C action: C, yes, right
Loss:  [0.0077455]
Policy - obs: C action: C, yes, right
Loss:  [0.00744226]
Policy - obs: A action: A, yes, right
Loss:  [0.00217394]
Policy - obs: B action: B, yes, right
Loss:  [0.00690658]
Policy - obs: C action: C, yes, right
Loss:  [0.00729281]
Policy - obs: A action: A, yes, right
Loss:  [0.00219843]
Policy - obs: B action: B, yes, right
Loss:  [0.00687765]
Policy - obs: B action: B, yes, right
Loss:  [0.00668574]
Policy - obs: C action: C, yes, right
Loss:  [0.00726891]
Policy - obs: A 

KeyboardInterrupt: 